# Chapter 13:🌐 HTTP & API Fundamentals

Welcome to this comprehensive guide on HTTP protocols and API interactions! This notebook covers everything from basic requests to advanced API features with practical examples you can run yourself.

## 📚 What You'll Learn
- HTTP request/response cycle
- Working with REST APIs
- Authentication methods
- Error handling and rate limiting
- Real-time data collection
- Combining APIs with web scraping

---

## 🔗 HTTP Basics & First API Call

HTTP (HyperText Transfer Protocol) is the foundation of data communication on the web. APIs (Application Programming Interfaces) allow different software systems to communicate with each other using HTTP protocols.

### HTTP Methods Overview:
- **GET**: Retrieve data from server
- **POST**: Send data to server
- **PUT**: Update existing resource
- **DELETE**: Remove resource

Let's make our first API call to a jokes API:

In [1]:
import requests

# Example: Get a random joke from the "Official Joke API"
url = "https://official-joke-api.appspot.com/random_joke"
response = requests.get(url)  # Send GET request

# Check status code (200 means success)
print("Status Code:", response.status_code)
print("Status Meaning:", "✅ Success" if response.status_code == 200 else "❌ Error")

# Get JSON data
data = response.json()
print("\n📝 Joke:", data['setup'])
print("🎭 Punchline:", data['punchline'])

Status Code: 200
Status Meaning: ✅ Success

📝 Joke: Bad at golf?
🎭 Punchline: Join the club.


## 📊 Understanding API Responses

APIs can return different data formats. The most common are:

1. **JSON** (JavaScript Object Notation) - Most common for APIs
2. **XML** (Extensible Markup Language) - Older but still used
3. **Plain Text** - Simple responses

Let's explore another API that returns JSON data:

In [2]:
# Dog CEO API - Get random dog images
import requests

url = "https://dog.ceo/api/breeds/image/random"
response = requests.get(url)
data = response.json()

print("Status Code:", response.status_code)
print("Full Response:", data)
print("🐶 Random Dog Image URL:", data["message"])
print("Request Status:", data["status"])

Status Code: 200
Full Response: {'message': 'https://images.dog.ceo/breeds/greyhound-italian/n02091032_6037.jpg', 'status': 'success'}
🐶 Random Dog Image URL: https://images.dog.ceo/breeds/greyhound-italian/n02091032_6037.jpg
Request Status: success


## 🔧 Working with API Parameters

Most APIs accept parameters to customize responses. Parameters can be:
- Added directly to the URL: `?key=value&another=value`
- Passed as a dictionary in `requests.get()`

### Example: Getting Multiple Results

In [3]:
import requests

# Get 3 dog images instead of just 1
url = "https://dog.ceo/api/breeds/image/random/3"
response = requests.get(url)

data = response.json()
print("🐕 Dog Images URLs:")
for i, img_url in enumerate(data['message'], 1):
    print(f"{i}. {img_url}")

🐕 Dog Images URLs:
1. https://images.dog.ceo/breeds/pointer-germanlonghair/hans2.jpg
2. https://images.dog.ceo/breeds/groenendael/n02105056_7381.jpg
3. https://images.dog.ceo/breeds/mastiff-bull/n02108422_1939.jpg


## 📑 Pagination

Many APIs limit how much data they return at once and use pagination to split results across multiple pages.

### Example: Working with Paginated Data

In [4]:
import requests

url = "https://jsonplaceholder.typicode.com/posts"
params = {"_page": 1, "_limit": 5}  # Page 1, 5 results per page

response = requests.get(url, params=params)
data = response.json()

print(f"📄 Page {params['_page']} Results:")
for post in data:
    print(f"Post {post['id']}: {post['title']}")

📄 Page 1 Results:
Post 1: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Post 2: qui est esse
Post 3: ea molestias quasi exercitationem repellat qui ipsa sit aut
Post 4: eum et est occaecati
Post 5: nesciunt quas odio


## 🔐 API Authentication

Many APIs require authentication to control access and track usage. Common methods:

1. **API Keys** - Simple token added to requests
2. **OAuth** - Secure authorization framework
3. **Bearer Tokens** - Token-based authentication

### Example: Using an API Key (OpenWeatherMap)

In [5]:
import requests

API_KEY = "Your api key"  # Example key (replace with your own)
url = "https://api.openweathermap.org/data/2.5/weather"
params = {"q": "London", "appid": API_KEY, "units": "metric"}

response = requests.get(url, params=params)
data = response.json()

if response.status_code == 200:
    print(f"🌤️ Weather in {data['name']}:")
    print(f"Temperature: {data['main']['temp']}°C")
    print(f"Conditions: {data['weather'][0]['description']}")
    print(f"Humidity: {data['main']['humidity']}%")
else:
    print(f"❌ Error: {data.get('message', 'Unknown error')}")

❌ Error: Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.


## ⚠️ Error Handling & Rate Limiting

APIs can fail for various reasons. Proper error handling ensures your application remains stable.

### Common API Errors:
- `400 Bad Request` - Invalid parameters
- `401 Unauthorized` - Invalid or missing authentication
- `403 Forbidden` - Authenticated but not authorized
- `404 Not Found` - Resource doesn't exist
- `429 Too Many Requests` - Rate limit exceeded
- `500 Internal Server Error` - Server-side issue

### Example: Comprehensive Error Handling

In [6]:
import requests

url = "https://catfact.ninja/facts"

try:
    response = requests.get(url, timeout=5)  # 5 second timeout
    response.raise_for_status()  # Raise error for bad status codes (4xx/5xx)
    
    data = response.json()
    print("✅ Success:")
    for i, fact in enumerate(data['data'][:3], 1):  # Show first 3 facts
        print(f"{i}. {fact['fact']}")
        
except requests.exceptions.Timeout:
    print("⏳ Request timed out")
except requests.exceptions.HTTPError as e:
    print(f"❌ HTTP Error: {e}")
except requests.exceptions.RequestException as e:
    print(f"⚠️ Other Error: {e}")

✅ Success:
1. Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.
2. When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.
3. The technical term for a cat’s hairball is a “bezoar.”


## 🚦 Rate Limiting

APIs often limit how many requests you can make to prevent abuse. Always check the API documentation for rate limits.

### Example: Respecting Rate Limits

In [7]:
import requests
import time

facts = []
url = "https://catfact.ninja/facts"

for i in range(3):  # Make 3 requests
    print(f"Request {i+1}...")
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        facts.extend([item['fact'] for item in data['data']])
    else:
        print(f"Error on request {i+1}: {response.status_code}")
    
    time.sleep(1)  # Wait 1 second between requests

print(f"\n📋 Collected {len(facts)} cat facts!")

Request 1...
Request 2...
Request 3...

📋 Collected 30 cat facts!


## 🗃️ Building Datasets from APIs

APIs are excellent sources for creating datasets. Let's build a dataset of cat facts:

In [13]:
import requests
import pandas as pd
import time
from requests.exceptions import RequestException

def fetch_cat_facts(goal_facts=50):
    """
    Fetches a specified number of cat facts from the Cat Fact API.
    """
    facts_list = []
    url = "https://catfact.ninja/facts"
    page = 1
    
    while len(facts_list) < goal_facts:
        params = {"limit": 10, "page": page}
        
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()  # Raises HTTPError for bad responses (4xx or 5xx)
            data = response.json()
            
            # Check for data
            if not data.get('data'):
                print("No more facts available.")
                break
                
            facts = [fact.get('fact') for fact in data['data']]
            facts_list.extend(facts)
            
            print(f"📄 Page {page}: Collected {len(facts_list)} facts so far.")
            
            # Check if we've reached the last page to avoid unnecessary requests
            if page >= data.get('last_page', page):
                print("Reached the last page of facts on the API.")
                break
            
            page += 1
            time.sleep(1)  # Respect API rate limits
            
        except RequestException as e:
            print(f"An error occurred on page {page}: {e}")
            break
        except KeyError as e:
            print(f"JSON parsing error: Missing key {e} on page {page}.")
            break
            
    return facts_list

def save_facts_to_csv(facts, filename="cat_facts.csv"):
    """
    Saves a list of facts to a CSV file.
    """
    df = pd.DataFrame(facts, columns=["fact"])
    df.to_csv(filename, index=False)
    print(f"✅ Saved {len(df)} cat facts to {filename}")
    print("\nSample facts:")
    print(df.head().to_string(index=False))

if __name__ == "__main__":
    collected_facts = fetch_cat_facts(goal_facts=50)
    save_facts_to_csv(collected_facts)

📄 Page 1: Collected 10 facts so far.
📄 Page 2: Collected 20 facts so far.
📄 Page 3: Collected 30 facts so far.
📄 Page 4: Collected 40 facts so far.
📄 Page 5: Collected 50 facts so far.
✅ Saved 50 cat facts to cat_facts.csv

Sample facts:
                                                                                                                                          fact
                            Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.
                                             When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.
                                                                                        The technical term for a cat’s hairball is a “bezoar.”
                                                                                                        A group of cats is called a “clowder.”
A cat can’t climb head first down a tree becaus

## ⚡ Real-Time API Data

Some APIs provide real-time or frequently updated data. Let's track cryptocurrency prices:

In [14]:
import requests
import pandas as pd
import time
from datetime import datetime

url = "https://api.coingecko.com/api/v3/simple/price"
params = {"ids": "bitcoin,ethereum", "vs_currencies": "usd"}
records = []

print("📊 Starting crypto price tracking...")
print("Press Ctrl+C to stop\n")

try:
    for i in range(5):  # Get 5 readings
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            btc_price = data['bitcoin']['usd']
            eth_price = data['ethereum']['usd']
            
            print(f"{timestamp} | BTC: ${btc_price} | ETH: ${eth_price}")
            
            records.append({
                "timestamp": timestamp,
                "btc_usd": btc_price,
                "eth_usd": eth_price
            })
        else:
            print(f"⚠️ API Error: {response.status_code}")
        
        time.sleep(5)  # Wait 5 seconds between requests
        
except KeyboardInterrupt:
    print("\nStopped by user")

# Save to CSV
if records:
    df = pd.DataFrame(records)
    df.to_csv("crypto_prices.csv", index=False)
    print(f"\n✅ Saved {len(df)} records to crypto_prices.csv")

📊 Starting crypto price tracking...
Press Ctrl+C to stop

2025-08-30 18:46:53 | BTC: $108916 | ETH: $4377.5
2025-08-30 18:46:58 | BTC: $108916 | ETH: $4377.5
2025-08-30 18:47:03 | BTC: $108916 | ETH: $4377.5
2025-08-30 18:47:08 | BTC: $108916 | ETH: $4377.5
2025-08-30 18:47:13 | BTC: $108916 | ETH: $4377.5

✅ Saved 5 records to crypto_prices.csv


## 🤝 API + Web Scraping Hybrid

Sometimes you need to combine API data with web scraping to get complete information:

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.exceptions import RequestException
import logging

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
NASA_API_URL = "https://images-api.nasa.gov/search"

def get_image_details(session, item_data, nasa_url):
    """
    Scrapes a specific NASA page for a description.
    Uses the session object to reuse connections.
    """
    try:
        page_response = session.get(nasa_url, timeout=10)
        page_response.raise_for_status()
        soup = BeautifulSoup(page_response.text, "html.parser")
        
        description = "No description found"
        # Try to find description in common meta tags or elements
        meta_desc = soup.find("meta", attrs={"name": "description"})
        if meta_desc:
            description = meta_desc.get("content", "No description found")
        
        # Consider an alternative search if meta tag is not present
        if description == "No description found":
            # Example of trying to find it in a different element
            p_desc = soup.find("p", class_="description-class-here") # Replace with actual class
            if p_desc:
                description = p_desc.text.strip()
                
        # Truncate and clean description
        clean_desc = description.replace("\n", " ").strip()
        final_description = (clean_desc[:200] + "...") if len(clean_desc) > 200 else clean_desc
        
        return final_description
    except RequestException as e:
        logging.error(f"Failed to scrape {nasa_url}: {e}")
        return "Scraping failed."

def main():
    """
    Main function to orchestrate the data collection and saving.
    """
    params = {"q": "moon", "media_type": "image"}
    records = []
    
    try:
        # Use a session for persistent connections
        with requests.Session() as session:
            response = session.get(NASA_API_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            items = data.get('collection', {}).get('items', [])
            
            # Use enumerate for a clearer loop with an index
            for i, item in enumerate(items[:5]): # Use a reasonable limit
                # Safely get data using .get() to prevent errors
                item_data = item.get('data', [{}])[0]
                title = item_data.get('title', 'No Title')
                nasa_url = item.get('links', [{}])[0].get('href', None)
                
                # Check if we can get description directly from API
                api_description = item_data.get('description', None)
                if api_description:
                    description = (api_description[:200] + "...") if len(api_description) > 200 else api_description
                    logging.info(f"✅ Collected from API: {title}")
                elif nasa_url:
                    # If not, scrape the individual page
                    description = get_image_details(session, item_data, nasa_url)
                    logging.info(f"✅ Scraped details for: {title}")
                else:
                    description = "No URL found"
                    logging.warning(f"Item {i+1} has no URL.")
                
                records.append({
                    "title": title,
                    "url": nasa_url,
                    "description": description
                })
                
                time.sleep(0.5) # Be polite to the server with a short delay

    except RequestException as e:
        logging.error(f"An error occurred during API call: {e}")
        return
    except (KeyError, IndexError) as e:
        logging.error(f"Failed to parse JSON response: {e}")
        return
    
    # Save results to a CSV file
    if records:
        df = pd.DataFrame(records)
        df.to_csv("nasa_images.csv", index=False)
        logging.info(f"\n📊 Saved {len(df)} records to nasa_images.csv")
        logging.info("\nSample data:\n" + df.head().to_string(index=False))

if __name__ == "__main__":
    main()

2025-08-30 18:48:51,989 - INFO - ✅ Collected from API: Nearside of the Moon
2025-08-30 18:48:52,570 - INFO - ✅ Collected from API: Color of the Moon
2025-08-30 18:48:53,073 - INFO - ✅ Collected from API: Moon to Mars Transportation and Habitation
2025-08-30 18:48:53,576 - INFO - ✅ Collected from API: Moon to Mars Multidisciplinary Science
2025-08-30 18:48:54,079 - INFO - ✅ Collected from API: Moon to Mars Infrastructure
2025-08-30 18:48:54,596 - INFO - 
📊 Saved 5 records to nasa_images.csv
2025-08-30 18:48:54,605 - INFO - 
Sample data:
                                     title                                                                                                                                   url                                                                                                                                                                                                 description
                      Nearside of the Moon                                   

## 🧪 Practice Exercises

### Exercise 1: Basic API Call
Create a program that fetches 5 random dog images and prints their URLs.

### Exercise 2: Error Handling
Modify the weather API example to handle these error scenarios:
- Invalid city name
Invalid API key
- Network timeout

### Exercise 3: Data Collection
Create a dataset of 30 facts from the Cat Facts API with proper pagination handling.

### Exercise 4: Real-time Monitoring
Create a program that tracks cryptocurrency prices every 10 seconds for 1 minute and calculates the average price for each coin.

## 🏆 Main Challenge: API Dashboard

Create a simple dashboard that displays:
1. Current weather in 3 cities of your choice
2. 5 random cat facts
3. Current prices of Bitcoin and Ethereum
4. A random dog image

**Bonus:** 
- Add error handling for all API calls
- Implement a refresh button to update all data
- Save the collected data to files with timestamps

## ✅ Conclusion

You've learned how to:
- Make HTTP requests to various APIs
- Handle different response formats and status codes
- Work with API parameters and pagination
- Implement authentication and rate limiting
- Build datasets from API data
 Combine APIs with web scraping
- Create real-time data collection systems

### 📚 Further Learning
- Explore API documentation for services you use
- Learn about OAuth authentication
- Experiment with WebSockets for real-time communication
- Try building a frontend interface for your API calls

Happy coding! 🚀